In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# !gdown --fuzzy "https://drive.google.com/file/d/12ycYSzLIG253AFN35Y6qoyf9wtkOjakp/view?usp=sharing"

In [4]:
# !tar -zxvf "./2017-01-trnmted.tgz"
# !tar -zxvf "./2017-01-trnmted/texts/DeEnItNlRo/DeEnItNlRo/DeEnItNlRo-DeEnItNlRo.tgz"
# !mv ./DeEnItNlRo-DeEnItNlRo ./texts
# !mkdir ./clean_texts
# !mkdir ./tokenizers
# !rm "./2017-01-trnmted.tgz"
# !rm -rf "./2017-01-trnmted"

In [5]:
# import data
# from pathlib import Path

# data.convert_files(Path('./texts'), Path('./clean_texts'))
# data.train_tokenizers(Path('./clean_texts'), Path('./tokenizers'))

In [ ]:
from train_model import train_model, train_epoch

# train_model("./clean_texts", "./tokenizers", 5)

In [6]:
from pathlib import Path

import torch
from sacrebleu.metrics import BLEU
from tokenizers import Tokenizer
from tqdm.auto import trange, tqdm

from data import TranslationDataset
from decoding import translate, get_attn_mask
from model import TranslationModel

import wandb

In [7]:
data_dir = Path("./clean_texts")
tokenizer_path = Path("./tokenizers")
src_tokenizer = Tokenizer.from_file(str(tokenizer_path / "tokenizer_de.json"))
tgt_tokenizer = Tokenizer.from_file(str(tokenizer_path / "tokenizer_en.json"))

In [18]:
config = {
    "batch_size" : 12,
    "lr" : 3e-4,
    "max_len" : 128,  # might be enough at first
    "num_encoder_layers" : 1,
    "num_decoder_layers" : 1,
    "emb_size" : 256,
    "dim_feedforward" : 1024,
    "n_head" : 8,
    "dropout_prob" : 0.1,
}


train_dataset = TranslationDataset(
    data_dir / "train.de.txt",
    data_dir / "train.en.txt",
    src_tokenizer,
    tgt_tokenizer,
    max_len=128 #config["max_len"],
)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    collate_fn = train_dataset.collate_translation_data,
    # shuffle=True,
)

In [19]:
train_dataset[7]

(tensor([   2,  269, 4900,  500,  427,  214,  774,   15,  419,  998,  231,  405,
          726,  695,   17,    3]),
 tensor([   2,   42,   10,  185,  517,  175,  225, 1394,  743,  155, 8506,  246,
          168,   10,   82,  426,  280,  199,  188,   17,    3]))

In [13]:
len(train_dataloader)

12882

In [9]:
device = torch.device("cpu") # torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

src_pad_id = tgt_tokenizer.token_to_id("[PAD]")
tgt_pad_id = tgt_tokenizer.token_to_id("[PAD]")

model = TranslationModel(
    config["num_encoder_layers"],
    config["num_decoder_layers"],
    config["emb_size"],
    config["dim_feedforward"],
    config["n_head"],
    src_tokenizer.get_vocab_size(),
    tgt_tokenizer.get_vocab_size(),
    config["dropout_prob"],
    src_pad_id,
    tgt_pad_id,
    config["max_len"]
)


print("Total no. of model parameters:",
    "pytorch_total_params =", sum(p.numel() for p in model.parameters())
)
model.to(device);

Total no. of model parameters: pytorch_total_params = 24914224


In [32]:
from train_model import train_epoch, evaluate

num_epochs=1
optimizer = torch.optim.Adam(model.parameters(), config["lr"])
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    config["lr"],
    steps_per_epoch=len(train_dataloader),
    epochs=num_epochs,
    pct_start=0.1
)
CELoss = torch.nn.CrossEntropyLoss(ignore_index=tgt_pad_id)

min_val_loss = float("inf")

# for epoch in trange(1, num_epochs + 1):
#     val_loss = evaluate(model, train_dataloader, CELoss, device, src_tokenizer, tgt_tokenizer)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/12882 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [65]:
# src

In [69]:
for i in train_dataloader:
    for line in i["tgt"]:
        print(line)
    print(i["tgt"])
    break

tensor([[    2,   698,   175,   215,   467,    15,  2040,    17,     3,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1],
        [    2,   194,   159,    10,    82,  2691,    64,   589,  4596,   155,
           223,   147,  1869,   155,   451,   155,   197,  1899,  3318,    30,
            42,    10,    76,  2433,  5685,    17,     3,     1,     1,     1,
             1,     1,     1,     1],
        [    2,    42,   223,   426,  7699,   880,   325,   197,  3479,    15,
           163,    42,   358,   155,  2075,   209,   162,   175,   199,   147,
           516,  1936,  7305,   255,   246,    42,   336,   155,   399,   147,
           333,  1227,    17,     3],
        [    2,   194,    42,   399,   168,  1428,  4112,    15,  6879,   341,
            42,   442,   168,    17,     3,     1,     1,     1,     1,     1,
             1,  